# 🎯 Proyecto de Modelado Predictivo de Resultados de Fútbol (Soccer Dataset de Kaggle)

## ✅ Descripción del Proyecto
En este proyecto se realiza un análisis predictivo usando el dataset de fútbol disponible en Kaggle ([Soccer Database](https://www.kaggle.com/datasets/hugomathien/soccer)), que contiene resultados detallados de partidos de fútbol entre 2008 y 2016.

El objetivo principal es crear **dos tipos de modelos de aprendizaje supervisado**:
1. **Clasificación:** Predecir el resultado del partido (`match_result`: Victoria Local, Empate o Victoria Visitante).
2. **Regresión:** Predecir la diferencia de goles en un partido (`goal_difference`).

---

## ✅ Flujo de Trabajo

### 1. Descarga y Preparación de Datos
- Dataset descargado directamente desde Kaggle mediante `kagglehub`.
- Se utilizó el archivo `match.csv` que contiene los resultados de los partidos.

### 2. Creación de Variables Objetivo
- **Diferencia de goles (`goal_difference`):**
  
  \[
  \text{goal\_difference} = \text{home\_team\_goal} - \text{away\_team\_goal}
  \]

- **Resultado del partido (`match_result`):**
  - **Home Win:** Victoria del equipo local.
  - **Draw:** Empate.
  - **Away Win:** Victoria del equipo visitante.

### 3. Selección de Variables Predictoras
- Goles del equipo visitante (`away_team_goal`).
- Año del partido (`year`).

---

## ✅ Modelos Construidos

### 📌 Modelos de Clasificación
**Objetivo:** Predecir el resultado del partido.

1. **K-Nearest Neighbors Classifier (KNN)**
2. **Regresión Logística**

#### Métricas:
- **Accuracy** (precisión global).
- Reporte de clasificación (precisión, recall y F1-score).

#### 📋 Resultados:
##### 🔹 KNN Classifier:
- **Accuracy:** 56.6%
- **Reporte:**



In [19]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hugomathien/soccer")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/soccer


In [21]:
import pandas as pd
import sqlite3
import os

# Define the path to the database file
path_to_downloaded_dataset_dir = '/kaggle/input/soccer'
DB_PATH = os.path.join(path_to_downloaded_dataset_dir, 'database.sqlite')

# Conectar a la base de datos SQLite y cargar la tabla 'Match'
try:
    conn = sqlite3.connect(DB_PATH)
    df_match = pd.read_sql_query("SELECT * FROM Match;", conn)
    conn.close()
    print("DataFrame df_match loaded successfully.")
except sqlite3.Error as e:
    print(f"Error loading database tables: {e}")
    df_match = None # Ensure df_match is None if loading fails

# Mostrar las primeras filas para verificar que todo esté bien
if df_match is not None:
    print(df_match.head())
else:
    print("df_match could not be loaded.")

DataFrame df_match loaded successfully.
   id  country_id  league_id     season  stage                 date  \
0   1           1          1  2008/2009      1  2008-08-17 00:00:00   
1   2           1          1  2008/2009      1  2008-08-16 00:00:00   
2   3           1          1  2008/2009      1  2008-08-16 00:00:00   
3   4           1          1  2008/2009      1  2008-08-17 00:00:00   
4   5           1          1  2008/2009      1  2008-08-16 00:00:00   

   match_api_id  home_team_api_id  away_team_api_id  home_team_goal  ...  \
0        492473              9987              9993               1  ...   
1        492474             10000              9994               0  ...   
2        492475              9984              8635               0  ...   
3        492476              9991              9998               5  ...   
4        492477              7947              9985               1  ...   

    SJA   VCH   VCD   VCA   GBH   GBD   GBA   BSH   BSD   BSA  
0  4.00  1.6

In [22]:
# Crear variables objetivo
df_match['goal_difference'] = df_match['home_team_goal'] - df_match['away_team_goal']

def classify_result(diff):
    if diff > 0:
        return 'Home Win'
    elif diff < 0:
        return 'Away Win'
    else:
        return 'Draw'

df_match['match_result'] = df_match['goal_difference'].apply(classify_result)

# Variables predictoras
df_match['year'] = pd.to_datetime(df_match['date']).dt.year
X = df_match[['away_team_goal', 'year']].fillna(0)

# Variables objetivo
y_class = df_match['match_result']  # Clasificación
y_reg = df_match['goal_difference']  # Regresión

# División de datos
from sklearn.model_selection import train_test_split
X_train, X_test, y_train_class, y_test_class = train_test_split(X, y_class, test_size=0.2, random_state=42)
_, _, y_train_reg, y_test_reg = train_test_split(X, y_reg, test_size=0.2, random_state=42)

# Escalado
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modelos
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score

# Clasificación
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train_scaled, y_train_class)
y_pred_knn_class = knn_clf.predict(X_test_scaled)

log_clf = LogisticRegression(max_iter=1000)
log_clf.fit(X_train_scaled, y_train_class)
y_pred_log_class = log_clf.predict(X_test_scaled)

print("=== KNN Classifier ===")
print("Accuracy:", accuracy_score(y_test_class, y_pred_knn_class))
print(classification_report(y_test_class, y_pred_knn_class))

print("=== Logistic Regression ===")
print("Accuracy:", accuracy_score(y_test_class, y_pred_log_class))
print(classification_report(y_test_class, y_pred_log_class))

# Regresión
knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(X_train_scaled, y_train_reg)
y_pred_knn_reg = knn_reg.predict(X_test_scaled)

lr_reg = LinearRegression()
lr_reg.fit(X_train_scaled, y_train_reg)
y_pred_lr_reg = lr_reg.predict(X_test_scaled)

print("\n=== KNN Regressor ===")
print("MSE:", mean_squared_error(y_test_reg, y_pred_knn_reg))
print("R2:", r2_score(y_test_reg, y_pred_knn_reg))

print("=== Linear Regression ===")
print("MSE:", mean_squared_error(y_test_reg, y_pred_lr_reg))
print("R2:", r2_score(y_test_reg, y_pred_lr_reg))


=== KNN Classifier ===
Accuracy: 0.5660123171670516
              precision    recall  f1-score   support

    Away Win       0.53      0.74      0.62      1470
        Draw       0.30      0.19      0.24      1317
    Home Win       0.69      0.67      0.68      2409

    accuracy                           0.57      5196
   macro avg       0.51      0.53      0.51      5196
weighted avg       0.55      0.57      0.55      5196

=== Logistic Regression ===
Accuracy: 0.6339491916859122


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

    Away Win       0.68      0.75      0.71      1470
        Draw       0.00      0.00      0.00      1317
    Home Win       0.61      0.91      0.73      2409

    accuracy                           0.63      5196
   macro avg       0.43      0.55      0.48      5196
weighted avg       0.48      0.63      0.54      5196


=== KNN Regressor ===
MSE: 1.8655735180908393
R2: 0.4198814864680519
=== Linear Regression ===
MSE: 1.6648416514070326
R2: 0.4823010432369923


## ✅ Conclusiones Finales

- El mejor modelo de **clasificación** fue la **Regresión Logística**, alcanzando un **63.4% de precisión (accuracy)** general. Sin embargo, mostró limitaciones importantes:
  - No logró predecir correctamente los empates (`Draw`), como se refleja en la precisión y recall de 0 para esa clase.
  - Esto indica que con las variables actuales (goles del visitante y año del partido), el modelo no tiene suficiente información para distinguir los empates.

- El mejor modelo de **regresión** fue la **Regresión Lineal**, obteniendo:
  - Un **MSE (Error Cuadrático Medio)** más bajo: **1.66**.
  - Un mejor **R² Score**: **0.48**, lo que indica un ajuste moderado al predecir la diferencia de goles.

- En ambas tareas (clasificación y regresión), los modelos basados en K-Nearest Neighbors (KNN) tuvieron un rendimiento inferior comparado con los modelos lineales.

---

### 🔑 Reflexión Final:
- Las variables usadas en este proyecto fueron muy básicas (goles del visitante y año), lo cual limita la capacidad predictiva, sobre todo para casos como el empate.
- A pesar de esto, el proyecto demuestra cómo aplicar técnicas de Machine Learning tanto para **clasificación** como para **regresión** en un contexto deportivo.
- Incorporar más variables (estadísticas de equipos, ratings, posesión, localía, entre otras) mejoraría considerablemente los resultados.
- También se pueden probar otros modelos avanzados como Random Forest, XGBo
